In [2]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

### Modelling Average Electricity Demand

In [ ]:
profile_df = pd.read_excel('data/demand/electricity-usage.xlsx', index_col=0)
postcode_demand_df = pd.read_excel(
    'data/demand/Energy-Usage-by-Postcode.xlsx', sheet_name='Usage by Postcode', skiprows=1)
residential_usage = postcode_demand_df[
    (postcode_demand_df['Data Type'] == 'Energy per customer per day (kWh)') &
    (postcode_demand_df['Data Type'] == 'Energy per customer per day (kWh)') &
    (postcode_demand_df['Customer/Consumption Type'] == 'Residential')
].set_index(['Year', 'Postcode'])

In [ ]:
MONTH_COLUMNS = ['January',	'February',	'March', 'April',	'May',	'June',
                 'July',	'August',	'September',	'October',	'November',	'December']
postcode = 4101


normalised_profile = profile_df / profile_df.sum()

postcode_usage_profile = normalised_profile.multiply(
    residential_usage.loc[2024, postcode][MONTH_COLUMNS])
apartment_usage_profile = postcode_usage_profile * .79
apartment_usage_profile.to_csv('data/processed/4101_monthly_usage_profile.csv')

In [158]:
x = pd.concat([profile_df['January'], apartment_usage_profile['January'],
              normalised_profile['January']], axis=1)
x.columns = ['Queensland Profile', '4104 Profile',
             'Normalised Queensland Profile']
x.reset_index()
x.to_csv('x.csv')

### Getting hourly emissions intensity

In [17]:
df = pd.read_csv('data/grid-generation/20250428 Queensland.csv')
df = df[['date', 'Emissions Intensity - kgCO₂e/MWh', 'Price - AUD/MWh']]
df['date'] = pd.to_datetime(df['date'])
daily_ci = df.groupby(df.date.dt.hour).mean()
daily_ci.index.name = 'hour'
daily_ci = daily_ci.rename(columns={
                           'Emissions Intensity - kgCO₂e/MWh': 'Emissions Intensity - kgCO₂e/KWh'}, errors='raise')
daily_ci['Emissions Intensity - kgCO₂e/KWh'] = daily_ci['Emissions Intensity - kgCO₂e/KWh'] / 1000
daily_ci = daily_ci.drop('date', axis=1)
daily_ci.to_csv('data/processed/grid_ci.csv')

## Construct demand dataset

In [173]:
import pandas as pd
import numpy as np
from calendar import monthrange
from datetime import datetime

# Assume df is your (24, 12) DataFrame with index Hour, columns January...December
apartment_usage_profile.columns = [col.strip()
                                   for col in apartment_usage_profile.columns]

# Build a daily profile interpolated smoothly
months = apartment_usage_profile.columns.tolist()
month_to_day = {
    month: datetime(2013, i+1, 15).timetuple().tm_yday  # center at 15th
    for i, month in enumerate(months)
}

# Day of year range
days_of_year = np.arange(1, 366)  # Non-leap year

# Initialize result container
interpolated_profiles = np.zeros((len(days_of_year), 24))

# Loop over each hour and interpolate over day of year
for hour in range(24):
    # Extract hourly values across months
    values = apartment_usage_profile.loc[hour].values
    centers = np.array(list(month_to_day.values()))

    # Interpolate across the full year using cubic spline or cosine weights
    interpolated = np.interp(
        days_of_year,
        centers,
        values,
        period=365  # to wrap around the year
    )

    interpolated_profiles[:, hour] = interpolated

# Now expand this into hourly series (365 days × 24 hours)
hourly_values = interpolated_profiles.repeat(24, axis=0).reshape(-1)[:8760]

# Apply hourly pattern
pattern = np.tile(np.arange(24), 365)
hourly_profile = np.zeros(8760)

for h in range(24):
    hourly_profile[pattern == h] = interpolated_profiles[:, h]

# Build datetime index
dt_index = pd.date_range("2013-01-01", "2013-12-31 23:00", freq="h")
hourly_demand = pd.Series(
    hourly_profile, index=dt_index, name="Demand (KWh)")
hourly_demand.index.name = 'Datetime'
hourly_demand.to_csv('data/processed/demand_timeseries.csv')